In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import StandardScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


In [2]:
random_state=42
best_models = {}

In [3]:
# Read in the data
df = pd.read_csv('../Data/FS_combined_skin_cancer.csv')
df

,smoke,drink,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,...,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,biopsed
0,False,False,55,False,0,True,True,True,True,3.0,...,6.0,5.0,1,True,True,False,True,True,True,True
1,False,True,79,False,1,True,False,False,False,1.0,...,5.0,5.0,1,True,True,False,False,True,True,True
2,False,True,52,False,0,False,True,True,True,3.0,...,15.0,10.0,1,False,True,False,True,True,True,True
3,False,False,74,True,0,False,False,False,False,1.0,...,15.0,10.0,1,True,True,True,False,True,True,True
4,False,True,58,True,0,True,True,True,True,1.0,...,9.0,7.0,1,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,False,False,23,True,0,False,True,True,True,0.0,...,0.0,0.0,0,False,False,False,False,False,False,False
1701,False,False,27,False,0,False,False,True,True,0.0,...,0.0,0.0,0,False,False,False,False,False,False,False
1702,True,True,23,False,1,False,False,True,True,0.0,...,0.0,0.0,0,False,False,False,False,False,False,False
1703,False,False,23,True,0,False,False,True,False,0.0,...,0.0,0.0,0,False,False,False,False,False,False,False


# New Section

In [4]:
from imblearn.under_sampling import ClusterCentroids, TomekLinks
from imblearn.over_sampling import RandomOverSampler

def splitting_data(df, sampling):
    X = df.drop(['diagnostic'], axis=1)
    y = df['diagnostic']

    if sampling == 'none':
        return X, y
    elif sampling == 'SMOTEENN':
        sampler = SMOTEENN(random_state=random_state)
    elif sampling == 'SMOTE':
        sampler = SMOTE(random_state=random_state)
    elif sampling == 'under':
        sampler = RandomUnderSampler(random_state=random_state)
    elif sampling == 'over':
        sampler = RandomOverSampler(random_state=random_state)
    elif sampling == 'cluster_centroids':
        sampler = ClusterCentroids(random_state=random_state)
    elif sampling == 'tomek_links':
        sampler = TomekLinks()

    X_resampled, y_resampled = sampler.fit_resample(X, y)
    return X_resampled, y_resampled


In [5]:

def training(X_train, y_train):
    # Create a KNN classifier with 5 neighbors
    LGBM = AdaBoostClassifier()
    # Fit the classifier to the data
    LGBM.fit(X_train, y_train)
    return LGBM

In [6]:
def best_model(modelName, accuracy, precision, recall, f1):
    best_models[modelName] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [7]:
def predict(modleName,LGBM, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = LGBM.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cr=classification_report(y_test, y_pred, output_dict=True)
    precision = cr['weighted avg']['precision']
    recall = cr['weighted avg']['recall']
    f1 = cr['weighted avg']['f1-score']
    best_model(modleName,accuracy,precision,recall,f1)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def optimize_with_grid(X_train, y_train, random_state=None):
    # Define a pipeline that first scales the data and then applies the classifier
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('adaboost', AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state), random_state=random_state))
    ])

    # Define the parameter grid to search
    param_grid = {
        'adaboost__n_estimators': [50, 100, 200],  # Number of base estimators to use
        'adaboost__learning_rate': [0.01, 0.1, 1],  # Learning rate shrinks the contribution of each classifier
    }

    # Create the GridSearchCV object
    adaboost_cv = GridSearchCV(pipe, param_grid, cv=5, verbose=1, n_jobs=-1)

    # Perform the grid search on the provided data
    adaboost_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = adaboost_cv.best_params_
    best_score = adaboost_cv.best_score_
    best_estimator = adaboost_cv.best_estimator_
    print("Best Parameters:", best_params)
    print("Best Score:", best_score)

    return best_estimator

# Example usage (ensure you have defined X_train, y_train, and random_state before calling this function)
# best_adaboost_model = optimize_with_grid(X_train, y_train, random_state=42)


<h1> LGBM on original data with optimization </h1>

In [9]:
# using function with no sampling
X, y= splitting_data(df, 'none')
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1494
0     211
Name: diagnostic, dtype: int64


In [11]:
LGBM1 = training(X_train, y_train)
y_pred = predict('original',LGBM1, X_test, y_test)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       1.00      1.00      1.00       304

    accuracy                           1.00       341
   macro avg       1.00      1.00      1.00       341
weighted avg       1.00      1.00      1.00       341



In [12]:
best_LGBM1 = optimize_with_grid(X_train, y_train)
prediction = predict('original_grid',best_LGBM1, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 0.01, 'adaboost__n_estimators': 50}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       1.00      1.00      1.00       304

    accuracy                           1.00       341
   macro avg       1.00      1.00      1.00       341
weighted avg       1.00      1.00      1.00       341



<h1> LGBM using SMOTE sampling </h1>

In [13]:
X,y = splitting_data(df, 'SMOTE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1494
0    1494
Name: diagnostic, dtype: int64


In [15]:
LGBM2 =training(X_train, y_train)
y_pred = predict('SMOTE',LGBM2, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       284
           1       1.00      1.00      1.00       314

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [16]:
best_LGBM2 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTE_grid',best_LGBM2, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 0.01, 'adaboost__n_estimators': 50}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       284
           1       1.00      1.00      1.00       314

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> LGBM using SMOTEENN sampling </h1>

In [17]:
X,y = splitting_data(df, 'SMOTEENN')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    1494
1    1494
Name: diagnostic, dtype: int64


In [19]:
LGBM3 =training(X_train, y_train)
y_pred = predict('SMOTEENN',LGBM3, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       306
           1       1.00      1.00      1.00       292

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [20]:
# from joblib import dump
# dump(LGBM3,'/content/LGBM_SMOTEENN.joblib')

In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
best_LGBM3 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTEENN_grid',best_LGBM3, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 0.01, 'adaboost__n_estimators': 50}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       306
           1       1.00      1.00      1.00       292

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> DT on Random undersampling </h1>

In [23]:
X,y = splitting_data(df, 'under')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    211
1    211
Name: diagnostic, dtype: int64


In [25]:
LGBM4 =training(X_train, y_train)
y_pred = predict('undersampling',LGBM4, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



In [26]:
best_LGBM4 = optimize_with_grid(X_train, y_train)
prediction = predict('undersampling_grid',best_LGBM4, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 0.01, 'adaboost__n_estimators': 100}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



<h1> DT on Random Oversampling </h1>

In [27]:
X,y = splitting_data(df, 'over')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    1494
0    1494
Name: diagnostic, dtype: int64


In [29]:
LGBM5 =training(X_train, y_train)
y_pred = predict('oversampling',LGBM5, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       1.00      1.00      1.00       306

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



In [30]:
best_LGBM5 = optimize_with_grid(X_train, y_train)
prediction = predict('oversampling_grid',best_LGBM5, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 0.01, 'adaboost__n_estimators': 50}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       292
           1       1.00      1.00      1.00       306

    accuracy                           1.00       598
   macro avg       1.00      1.00      1.00       598
weighted avg       1.00      1.00      1.00       598



<h1> DT on Cluster Centroids </h1>

In [31]:
X,y = splitting_data(df, 'cluster_centroids')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/Users/maryam/opt/anaconda3/lib/python3.9/site-packages/imblearn/under_sampling/_prototype_generation/_cluster_centroids.py:178: ConvergenceWarning: Number of distinct clusters (204) found smaller than n_clusters (211). Possibly due to duplicate points in X.
  self.estimator_.fit(_safe_indexing(X, target_class_indices))


In [32]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    211
1    211
Name: diagnostic, dtype: int64


In [33]:
LGBM6 =training(X_train, y_train)
y_pred = predict('cluster_centroids',LGBM6, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



In [34]:
best_LGBM6 = optimize_with_grid(X_train, y_train)
prediction = predict('cluster_centroids_grid',best_LGBM6, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 1, 'adaboost__n_estimators': 50}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        50

    accuracy                           1.00        85
   macro avg       1.00      1.00      1.00        85
weighted avg       1.00      1.00      1.00        85



<h1> DT on Tomek Links </h1>

In [35]:
X,y = splitting_data(df, 'tomek_links')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    1494
0     211
Name: diagnostic, dtype: int64


In [37]:
LGBM7 =training(X_train, y_train)
y_pred = predict('tomek_links',LGBM7, X_test, y_test)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00       302

    accuracy                           1.00       341
   macro avg       1.00      1.00      1.00       341
weighted avg       1.00      1.00      1.00       341



In [38]:
best_LGBM7 = optimize_with_grid(X_train, y_train)
prediction = predict('tomek_links_grid',best_LGBM7, X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'adaboost__learning_rate': 0.01, 'adaboost__n_estimators': 50}
Best Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00       302

    accuracy                           1.00       341
   macro avg       1.00      1.00      1.00       341
weighted avg       1.00      1.00      1.00       341



In [39]:
best_model_df = pd.DataFrame.from_dict(best_models, orient='index')
best_model_df.sort_values(by='accuracy', ascending=False, inplace=True)
best_model_df

,accuracy,precision,recall,f1
original,1.0,1.0,1.0,1.0
original_grid,1.0,1.0,1.0,1.0
SMOTE,1.0,1.0,1.0,1.0
SMOTE_grid,1.0,1.0,1.0,1.0
SMOTEENN,1.0,1.0,1.0,1.0
SMOTEENN_grid,1.0,1.0,1.0,1.0
undersampling,1.0,1.0,1.0,1.0
undersampling_grid,1.0,1.0,1.0,1.0
oversampling,1.0,1.0,1.0,1.0
oversampling_grid,1.0,1.0,1.0,1.0
